# Biostat M280 Homework 2

**Due May 11 @ 11:59PM**


## Q1. Nonnegative Matrix Factorization

Nonnegative matrix factorization (NNMF) was introduced by [Lee and Seung (1999)](https://www.nature.com/articles/44565) as an analog of principal components and vector quantization with applications in data compression and clustering. In this homework we consider algorithms for fitting NNMF and (optionally) high performance computing using graphical processing units (GPUs).

<img src="./nnmf.png" width="500" align="center"/>

In mathematical terms, one approximates a data matrix $\mathbf{X} \in \mathbb{R}^{m \times n}$ with nonnegative entries $x_{ij}$ by a product of two low-rank matrices $\mathbf{V} \in \mathbb{R}^{m \times r}$ and $\mathbf{W} \in \mathbb{R}^{r \times n}$ with nonnegative entries $v_{ik}$ and $w_{kj}$. Consider minimization of the squared Frobenius norm
$$
	L(\mathbf{V}, \mathbf{W}) = \|\mathbf{X} - \mathbf{V} \mathbf{W}\|_{\text{F}}^2 = \sum_i \sum_j \left(x_{ij} - \sum_k v_{ik} w_{kj} \right)^2, \quad v_{ik} \ge 0, w_{kj} \ge 0,
$$
which should lead to a good factorization. Later in the course we will learn how to derive a majorization-minimization (MM) algorithm with iterative updates
$$
	v_{ik}^{(t+1)} = v_{ik}^{(t)} \frac{\sum_j x_{ij} w_{kj}^{(t)}}{\sum_j b_{ij}^{(t)} w_{kj}^{(t)}}, \quad \text{where } b_{ij}^{(t)} = \sum_k v_{ik}^{(t)} w_{kj}^{(t)},
$$
$$
	w_{kj}^{(t+1)} = w_{kj}^{(t)} \frac{\sum_i x_{ij} v_{ik}^{(t+1)}}{\sum_i b_{ij}^{(t+1/2)} v_{ik}^{(t+1)}}, \quad \text{where } b_{ij}^{(t+1/2)} = \sum_k v_{ik}^{(t+1)} w_{kj}^{(t)}
$$
that drive the objective $L^{(t)} = L(\mathbf{V}^{(t)}, \mathbf{W}^{(t)})$ downhill. Superscript $t$ indicates iteration number. Efficiency (both speed and memory) will be the most important criterion when grading this problem.


1. Implement the algorithm with arguments: $\mathbf{X}$ (data, each row is a vectorized image), rank $r$, convergence tolerance, and optional starting point.
```julia
function nnmf(
    X::Matrix{T},
    r::Integer;
    maxiter::Integer=1000, 
    tol::Number=1e-4,
    V::Matrix{T}=rand(T, size(X, 1), r),
    W::Matrix{T}=rand(T, r, size(X, 2))
    ) where T <: AbstractFloat
    # implementation
    # Output
    return V, W
end
```

0. Database 1 from the [MIT Center for Biological and Computational Learning (CBCL)](http://cbcl.mit.edu) reduces to a matrix $\mathbf{X}$ containing $m = 2,429$ gray-scale face images with $n = 19 \times 19 = 361$ pixels per face. Each image (row) is scaled to have mean and standard deviation 0.25.  
Read in the [`nnmf-2429-by-361-face.txt`](http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/nnmf-2429-by-361-face.txt) file, e.g., using [`readdlm()`](https://docs.julialang.org/en/stable/stdlib/io-network/#Base.DataFmt.readdlm-Tuple{Any,Char,Type,Char}) function, and display a couple sample images, e.g., using [ImageView.jl](https://github.com/JuliaImages/ImageView.jl) package.

0. Report the run times, using `@time`, of your function for fitting NNMF on the MIT CBCL face data set at ranks $r=10, 20, 30, 40, 50$. For ease of comparison (and grading), please start your algorithm with the provided $\mathbf{V}^{(0)}$ (first $r$ columns of [`V0.txt`](http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/V0.txt)) and $\mathbf{W}^{(0)}$ (first $r$ rows of [`W0.txt`](http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/W0.txt)) and stopping criterion
$$
	\frac{|L^{(t+1)} - L^{(t)}|}{|L^{(t)}| + 1} \le 10^{-4}.
$$

0. Choose an $r \in \{10, 20, 30, 40, 50\}$ and start your algorithm from a different $\mathbf{V}^{(0)}$ and $\mathbf{W}^{(0)}$. Do you obtain the same objective value and $(\mathbf{V}, \mathbf{W})$? Explain what you find.

0. For the same $r$, start your algorithm from $v_{ik}^{(0)} = w_{kj}^{(0)} = 1$ for all $i,j,k$. Do you obtain the same objective value and $(\mathbf{V}, \mathbf{W})$? Explain what you find.

0. Plot the basis images (rows of $\mathbf{W}$) at rank $r=50$. What do you find?

0. (Optional) Investigate the GPU capabilities of Julia. Report the speed gain of your GPU code over CPU code at ranks $r=10, 20, 30, 40, 50$. Make sure to use the same starting point as in part 2.

## 1. Implement the algorithm with arguments:  X

In [13]:
function mynnmf(X::Matrix{T}, r::Int, 
        tol::Float64 = 1e-4, maxiter = 1000,
        V::Matrix{T}=rand(T, size(X, 1), r),
        W::Matrix{T}=rand(T, r, size(X, 2)),
    ) where T <: AbstractFloat
    
    m = size(X, 1)
    n = size(X, 2)
    mn = m * n
    loss = 0
    # Iteration
    for iter = 1: maxiter    
        for k = 1:r
            for i = 1:m
                num = 0
                den = 0
                for j = 1:n 
                    #num = num + X[i, j]*W[k, j]
                    BLAS.gemm!('N', 'T', 1.0, X, W, 0.0, num)
                    b = 0
                    #for t = 1: r
                        #b = b + V[i, t]*W[t, j]
                    #end
                    # B = V * W
                      BLAS.gemm!('N', 'N', 1.0, V, W, 0.0, B)
                    #den = den + b*W[k, j]
                      BLAS.gemm!('N', 'T', 1.0, B, W, 0.0, den)  
                end 
                V[i, k] = V[i, k]*num/den
            end
        end
        for j = 1:n  
            for k = 1:r
                num1 = 0
                den1 = 0
                for i = 1:m
                    num1 = num1 +  X[i, j]*V[i, k]
                    b1 = 0
                    for t = 1:r
                        b1 = b1 + V[i, t]*W[t, j]
                    end
                    den1 = den1 + b1*V[i, k]                   
                end
                W[k, j] = W[k, j]*num1/den1
            end
        end
        # minimization of the squared Frobenius norm
        B = V * W
        diff = X - B
        lastloss = loss
        loss = vecnorm(diff)^2
         # convergency or not
        if abs(loss - lastloss) < tol
            break
        end
    end   
    #return
    return V, W
end

mynnmf (generic function with 5 methods)

In [ ]:
A = one(10,10)
B = one(10, 10)
BLAS.dot(10, A, 1, B, 1)

In [40]:
function MYnnmf(X::Matrix{T}, r::Int, 
        tol::Float64 = 1e-4, maxiter = 1000,
        V::Matrix{T}=rand(T, size(X, 1), r),
        W::Matrix{T}=rand(T, r, size(X, 2)),
    ) where T <: AbstractFloat
    
    m = size(X, 1)
    n = size(X, 2)
    mn = m * n
    num_V = zeros(V)
    den_V = zeros(V)
    num_W = zeros(W)
    den_W = zeros(W)
    B = V * W
    loss = vecnorm(X - B)^2
    # Iteration
    for iter = 1: maxiter
        BLAS.gemm!('N', 'T', 1.0, X, W, 0.0, num_V)
        BLAS.gemm!('N', 'T', 1.0, B, W, 0.0, den_V)
        for j = 1:r
            for i = 1:m
                V[i, j] = V[i, j]*(num_V[i, j] / den_V[i, j])
            end
        end
        BLAS.gemm!('N', 'N', 1.0, V, W, 0.0, B)
        
        BLAS.gemm!('T', 'N', 1.0, V, X, 0.0, num_W)
        BLAS.gemm!('T', 'N', 1.0, V, B, 0.0, den_W)
        for j = 1:n
            for i = 1:r
                 W[i, j] = W[i, j]*(num_W[i, j] / den_W[i, j])
            end
        end
        BLAS.gemm!('N', 'N', 1.0, V, W, 0.0, B)
        # convergency
        diff = X - B
        lastloss = loss
        loss = vecnorm(diff)^2
         # convergency or not
        if abs(loss - lastloss) < tol
            break
        end
    end
    return V, W
end

MYnnmf (generic function with 5 methods)

In [ ]:
mynnmf(X, 10)

In [42]:
#try nnmf
V10, W10 = MYnnmf(X, 10)


([0.0725194 0.00435389 … 0.146856 2.31878e-53; 0.0866178 0.00896423 … 0.14822 1.82669e-71; … ; 0.0350167 0.023332 … 0.0324174 0.100953; 0.126694 2.56738e-68 … 0.0749996 0.063186], [1.0876e-54 5.8935e-58 … 0.786389 0.242629; 0.443146 0.360416 … 2.30524 2.35853; … ; 2.0503e-30 4.3925e-51 … 3.55808e-58 1.7383e-74; 4.30787e-25 0.183829 … 8.16043e-8 1.52735e-27])

2429×10 Array{Float64,2}:
 0.00095815   0.0103217    …  0.258624   0.0148599    0.113121   
 2.02987e-5   0.0502429       0.304791   0.0243719    0.0782129  
 0.000274559  0.0168783       0.312746   0.0254831    0.0709939  
 0.0389389    1.25395e-6      0.280938   0.0715732    0.0824829  
 8.09569e-5   7.68997e-5      0.302344   0.0854243    0.0710568  
 1.44156e-5   0.0141628    …  0.337035   0.0804725    0.0577115  
 0.0575214    4.34102e-6      0.286825   0.132121     0.0584703  
 0.102875     0.000172083     0.0844729  0.365178     0.00367533 
 0.0931789    0.026979        0.0808984  0.41947      0.00472247 
 0.123543     0.00105517      0.0963775  0.422801     0.00482261 
 0.138481     0.00111706   …  0.0823581  0.190548     0.00236216 
 0.00385435   3.70107e-6      0.287682   0.0569465    0.118763   
 0.00017552   3.40456e-5      0.279995   0.00140316   0.0757149  
 ⋮                         ⋱                                     
 0.0163516    0.0494912       0.173784   0.0461157

In [41]:
# readin matrix X
path_X = "http://Hua-Zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/nnmf-2429-by-361-face.txt"
X = readdlm(download(path_X), ' ')

# readin matrix V0
path_V0 = "http://Hua-Zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/V0.txt"
V0full = readdlm(download(path_V0), ' ')

# readin matrix W0
path_W0 = "http://Hua-Zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/W0.txt"
W0full = readdlm(download(path_W0), ' ');

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6012k  100 6012k    0     0  2053k      0  0:00:02  0:00:02 --:--:-- 2053k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  948k  100  948k    0     0   655k      0  0:00:01  0:00:01 --:--:--  654k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  140k  100  140k    0     0   259k      0 --:--:-- --:--:-- --:--:--  259k


In [2]:
X

LoadError: [91mUndefVarError: X not defined[39m

LoadError: [91mMethodError: objects of type Array{Float64,2} are not callable
Use square brackets [] for indexing an Array.[39m

In [19]:
# Pkg.add("ImageView")
#Pkg.add("TestImages")
using ImageView, TestImages
imshow(reshape(X[1, :],19,19))


Dict{String,Any} with 4 entries:
  "gui"         => Dict{String,Any}(Pair{String,Any}("window", Gtk.GtkWindowLea…
  "roi"         => Dict{String,Any}(Pair{String,Any}("redraw", 37: "map(clim-ma…
  "annotations" => 3: "input-2" = Dict{UInt64,Any}() Dict{UInt64,Any} 
  "clim"        => 2: "CLim" = ImageView.CLim{Float64}(0.0, 0.93656) ImageView.…

In [6]:
rlist = 10:10:50;


## Q2. Linear Mixed Models

Consider a linear mixed effects model
$$
	y_i = \mathbf{x}_i^T \beta + \mathbf{z}_i^T \gamma + \epsilon_i, \quad i=1,\ldots,n,
$$
where $\epsilon_i$ are independent normal errors $N(0,\sigma_0^2)$, $\beta \in \mathbb{R}^p$ are fixed effects, and $\gamma \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \sigma_1^2 \mathbf{I}_q$) independent of $\epsilon_i$. 

0. Show that 
$$
    \mathbf{y} \sim N \left( \mathbf{X} \beta, \sigma_0^2 \mathbf{I}_n + \sigma_1^2 \mathbf{Z} \mathbf{Z}^T \right),
$$
where $\mathbf{y} = (y_1, \ldots, y_n)^T \in \mathbb{R}^n$, $\mathbf{X} = (\mathbf{x}_1, \ldots, \mathbf{x}_n)^T \in \mathbb{R}^{n \times p}$, and $\mathbf{Z} = (\mathbf{z}_1, \ldots, \mathbf{z}_n)^T \in \mathbb{R}^{n \times q}$. 

0. Write a function, with interface 
    ```julia
    logpdf_mvn(y::Vector, Z::Matrix, σ0::Number, σ1::Number),
    ```
that evaluates the log-density of a multivariate normal with mean $\mathbf{0}$ and covariance $\sigma_0^2 \mathbf{I} + \sigma_1^2 \mathbf{Z} \mathbf{Z}^T$ at $\mathbf{y}$. Make your code efficient in the $n \gg q$ case. 

0. Compare your result (both accuracy and timing) to the [Distributions.jl](http://distributionsjl.readthedocs.io/en/latest/multivariate.html#multivariate-normal-distribution) package using following data.  
    ```julia
    using BenchmarkTools, Distributions

    srand(280)
    n, q = 2000, 10
    Z = randn(n, q)
    σ0, σ1 = 0.5, 2.0
    Σ = σ1^2 * Z * Z.' + σ0^2 * I
    mvn = MvNormal(Σ) # MVN(0, Σ)
    y = rand(mvn) # generate one instance from MNV(0, Σ)

    # check you answer matches that from Distributions.jl
    @show logpdf_mvn(y, Z, σ0, σ1)
    @show logpdf(mvn, y)

    # benchmark
    @benchmark logpdf_mvn(y, Z, σ0, σ1)
    @benchmark logpdf(mvn, y)
    ```